<a href="https://colab.research.google.com/github/gmshroff/metaLearning2022/blob/main/project_data_code/market_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt 
import torch
from datetime import datetime
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
class TsDS(Dataset):
    def __init__(self, XL,yL,flatten=False,lno=None,long=True):
        self.samples=[]
        self.labels=[]
        self.flatten=flatten
        self.lno=lno
        self.long=long
        self.scaler = StandardScaler()
        for X,Y in zip(XL,yL):
            self.samples += [torch.tensor(X).float()]
            self.labels += [torch.tensor(Y)]
    def __len__(self):
        return sum([s.shape[0] for s in self.samples])
    def __getitem__(self, idx):
        if self.flatten: sample=self.samples[idx].flatten(start_dim=1)
        else: sample=self.samples[idx]
        if self.lno==None: label=self.labels[idx]
        elif self.long: label=self.labels[idx][:,self.lno].long()
        else: label=self.labels[idx][:,self.lno].float()
        return (sample,label)
    def fit(self,kind='seq'):
        if kind=='seq':
            self.lastelems=[torch.cat([s[:,-1,:] for s in self.samples],dim=0)]
            self.scaler.fit(torch.cat([le for le in self.lastelems],dim=0))            
        elif kind=='flat': self.scaler.fit(torch.cat([s for s in self.samples],dim=0))
    def scale(self,kind='flat',scaler=None):
        def cs(s):
            return (s.shape[0]*s.shape[1],s.shape[2])
        if scaler==None: scaler=self.scaler
        if kind=='seq':
            self.samples=[torch.tensor(scaler.transform(s.reshape(cs(s))).reshape(s.shape)).float() for s in self.samples]
            pass
        elif kind=='flat':
            self.samples=[torch.tensor(scaler.transform(s)).float() for s in self.samples]
    def unscale(self,kind='flat',scaler=None):
        def cs(s):
            return (s.shape[0]*s.shape[1],s.shape[2])
        if scaler==None: scaler=self.scaler
        if kind=='seq':
            self.samples=[torch.tensor(scaler.inverse_transform(s.reshape(cs(s))).reshape(s.shape)).float() for s in self.samples]
            pass
        elif kind=='flat':
            self.samples=[torch.tensor(scaler.inverse_transform(s)).float() for s in self.samples]

In [ ]:
files.upload()

In [ ]:
!mkdir /root/.kaggle

In [ ]:
!mv ./kaggle.json /root/.kaggle/.

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gmshroff/market-data

In [ ]:
!unzip market-data.zip

In [ ]:
!ls marketdata

In [ ]:
with open('./marketdata/train_cs_0_0_2.pickle','rb') as f: train_cs=pickle.load(f)

In [ ]:
train_cs.samples[0].shape

In [ ]:
with open('./marketdata/train_ds_0_0_2.pickle','rb') as f: train_ds=pickle.load(f)

In [ ]:
train_ds.samples[0][0][:,3]

In [ ]:
train_cs.labels[0][0]

In [ ]:
train_ds.labels[0][0]